In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
from pathlib2 import Path
import preprocess
from data_loader import create_dataset, create_inputs

In [ ]:
def prepare_data(data_dir, language):
    if language == "Chinese":
        train_path = data_dir / "train_cn.json"
        test_path = data_dir / "test_cn.json"
        dev_path = data_dir / "dev_cn.json"

    elif language == "English":
        train_path = data_dir / "train_en.json"
        test_path = data_dir / "test_en.json"
        dev_path = data_dir / "dev_en.json"

    else:
        raise ValueError("Language must be English or Chinese")

    if not os.path.isfile(test_path):
        test_path = None

    return train_path, dev_path, test_path

data_dir = "./dataset"
language = "Chinese"
train_path, dev_path, test_path = prepare_data(data_dir=Path(data_dir), language=language)
train_path, dev_path, test_path

In [ ]:
model_input, customer_labels, helpdesk_labels, _, embedding_size  = create_inputs(json_path=train_path, plm="BERT",
                                                max_len=512,
                                                is_train=True, language="Chinese")
inputs = (model_input, customer_labels, helpdesk_labels, [])
dataset, max_turn_number = create_dataset(inputs, "nugget", 10000)
a = list(dataset.take(1))
input_ids = a[0]["input_ids"]
input_mask = a[0]["input_mask"]
input_type_ids = a[0]["input_type_ids"]
sentence_ids = a[0]["sentence_ids"]
sentence_masks = a[0]["sentence_masks"]
inputs = {
        "input_ids": input_ids, "input_mask": input_mask, "input_type_ids": input_type_ids,
        "sentence_ids": sentence_ids, "sentence_masks": sentence_masks
    }
from model import Bert
plm = Bert(language=language, embedding_size=embedding_size)

x = plm(inputs)

from model import TransformerEncoder
encoder = TransformerEncoder(d_model=768, num_heads=8, d_ff=200, maximum_position_encoding=10000, rate=0.1, num_layers=2)
posemb = encoder.pos_encoding[:, :7, :]
x += posemb

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

sentence_masks = create_padding_mask(sentence_masks)

for i in range(encoder.num_layers):
    x = encoder.enc_layers[i](x, training=True, mask=sentence_masks)
    
x.shape

In [ ]:
res2=x
customer_indices = tf.range(start=0, delta=2, limit=max_turn_number)
helpdesk_indices = tf.range(start=1, delta=2, limit=max_turn_number)
sentence_masks = a[0]["sentence_masks"]
cres = tf.gather(res2, indices=customer_indices, axis=1)
hres = tf.gather(res2, indices=helpdesk_indices, axis=1)

cust_mask = tf.gather(sentence_masks, axis=1, indices=customer_indices)

customer_dense = tf.keras.layers.Dense(5, activation='gelu')
clogits = customer_dense(cres) * tf.cast(cust_mask[:, :, None], tf.float32)

cprob = tf.nn.softmax(clogits)
clabels = a[0]["customer_labels"]

In [ ]:
cprob.shape, clabels.shape

In [ ]:
squared_difference = tf.math.squared_difference(
    cprob, clabels, name=None
)
squared_difference

In [ ]:

squared_difference

In [ ]:
mid = tf.reduce_sum(squared_difference, axis=-1)
mid

In [ ]:
aa = -tf.experimental.numpy.log2(tf.math.sqrt(mid / 2))
aa   

In [ ]:
xx = aa * tf.cast(cust_mask, tf.float32)
xx

In [ ]:
bb = tf.math.count_nonzero(xx, axis=-1)
bb

In [ ]:
tf.math.divide(tf.reduce_sum(xx, axis=-1), tf.cast(bb,dtype=tf.float32))

In [ ]:
tf.reduce_mean(tf.math.divide(tf.reduce_sum(xx, axis=-1), tf.cast(bb,dtype=tf.float32)))

In [ ]:
tf.reduce_mean(aa, axis=-1)

In [ ]:
tf.reduce_mean(xx, axis=-1)

In [ ]:
r = tf.reduce_sum(mid, axis=-1)
from math import log2
-log2(tf.reduce_mean(tf.math.sqrt(r/2)))

In [ ]:
cus_rnss = tf.math.sqrt(tf.reduce_sum(tf.reduce_sum(squared_difference, axis=-1), axis=-1) / 2)
-tf.experimental.numpy.log2(cus_rnss)

In [ ]:
def squared_error(pred, truth):
    return ((pred - truth) ** 2)

cprob_numpy = cprob.numpy()
ctruth_numpy = clabels.numpy()
squared_error(cprob_numpy, ctruth_numpy) * cust_mask[:, :, None].numpy()